In [1]:
import random
import csv 
import cv2
import os
import numpy as np
from numpy import mean
from numpy import std
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.applications.vgg16 import VGG16
import tensorflow_addons as tfa
from tensorflow_addons.optimizers import AdamW
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models,layers
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg

In [2]:
base_dir = '../input/dataset-cedulas/DeformedNotes_AME'

In [3]:
def readImages( path, X_train, Y_train  ):
  print("Loading data...")

  files = [ f for f in listdir(path) if isfile(join(path,f)) ]
  files.sort()
  #print(files)

  print( len(files), "images..." )

  for i in range( len(files) ):		#  i = identificador ----- 0 - qnt de imagens
    img = cv2.imread(join(path,files[i]),0)
    img = cv2.resize(img, (150, 150))
    X_train.append( img )

    name = files[i].split("_")
    name = name[3].split(".")
    if( int(name[0]) == 1 ):
      Y_train.append( 0 )
    elif( int(name[0]) == 2 ):
      Y_train.append( 1 )
    elif( int(name[0]) == 5 ):
      Y_train.append( 2 )
    elif( int(name[0]) == 10 ):
      Y_train.append( 3 )
    elif( int(name[0]) == 20 ):
      Y_train.append( 4 )
    elif( int(name[0]) == 50 ):
      Y_train.append( 5 )
    elif( int(name[0]) == 100 ):
      Y_train.append( 6 )
    elif( int(name[0]) == 200 ):
      Y_train.append( 7 )
    else:
      print(".: ", i, " - ", name)

  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  print('X_data shape:', X_train.shape)
  print('Y_data shape:', Y_train.shape)
#  print('Y_data shape:', Y_train)

  # one hot encode target values
  Y_train = to_categorical(Y_train)
  print(Y_train.shape)
  print(Y_train[12])

  # cv2.imshow(img)

  # convert from integers to floats
  X_train = X_train.astype('float32')
  print(X_train.shape)
  
  # normalize to range 0-1
  X_train = X_train / 255.0
  print(X_train.shape)
  
  del files

  return X_train, Y_train

In [4]:
  X_train = []
  Y_train = []

  X_train, Y_train = readImages(base_dir, X_train, Y_train)
  print(X_train.shape)
  print(Y_train.shape)

Loading data...
16152 images...
X_data shape: (16152, 150, 150)
Y_data shape: (16152,)
(16152, 7)
[0. 0. 0. 0. 0. 1. 0.]
(16152, 150, 150)
(16152, 150, 150)
(16152, 150, 150)
(16152, 7)


In [5]:
X_train = X_train.reshape(-1, 150, 150, 1)

In [6]:
INPUT_SHAPE = (150, 150, 1)
NUM_CLASSES = 7

def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=INPUT_SHAPE))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(rate=0.25))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
model = define_model()
model.summary()

2022-09-02 16:57:33.138784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 16:57:33.148148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 16:57:33.148960: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 16:57:33.150271: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0

node zero
2022-09-02 16:57:33.151393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 16:57:33.152092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 16:57:33.633906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 16:57:33.634909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 16:57:33.635714: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node r

In [8]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
    print("Evaluating model...")
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    c = 1
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        print("fold ", c)
        # define model

        model = define_model()
        
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model

        history = model.fit(trainX, trainY, epochs=30, batch_size=8, validation_data=(testX, testY), verbose=1)
        
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=1)
        print('> %.3f' % (acc * 100.0))

        # stores scores
        scores.append(acc)
        histories.append(history)
        c += 1

    return scores, histories

In [9]:
print(X_train.shape)
print(Y_train.shape)

print("Model setting...")
define_model()

print("Training and evaluating model...")
scores, histories = evaluate_model(X_train, Y_train, n_folds=5)

print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))

(16152, 150, 150, 1)
(16152, 7)
Model setting...
Training and evaluating model...
Evaluating model...
fold  1


2022-09-02 16:57:34.695967: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1162890000 exceeds 10% of free system memory.
2022-09-02 16:57:36.042799: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1162890000 exceeds 10% of free system memory.
2022-09-02 16:57:37.010982: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-09-02 16:57:37.832454: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1616/1616 [==============================] - 15s 8ms/step - loss: 1.8227 - accuracy: 0.2336 - val_loss: 1.7906 - val_accuracy: 0.2510
Epoch 2/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.7941 - accuracy: 0.2484 - val_loss: 1.7800 - val_accuracy: 0.2491
Epoch 3/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.7778 - accuracy: 0.2589 - val_loss: 1.7634 - val_accuracy: 0.2696
Epoch 4/30
1616/1616 [==============================] - 13s 8ms/step - loss: 1.7541 - accuracy: 0.2734 - val_loss: 1.7423 - val_accuracy: 0.2804
Epoch 5/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.7075 - accuracy: 0.2973 - val_loss: 1.6740 - val_accuracy: 0.3132
Epoch 6/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.6286 - accuracy: 0.3326 - val_loss: 1.6055 - val_accuracy: 0.3265
Epoch 7/30
1616/1616 [==============================] - 13s 8ms/step - loss: 1.5242 - accuracy: 0.3839 - val_loss: 1.4958 - val_accuracy: 0.3

2022-09-02 17:04:01.815091: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1162890000 exceeds 10% of free system memory.
2022-09-02 17:04:03.147202: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1162890000 exceeds 10% of free system memory.


Epoch 1/30
1616/1616 [==============================] - 14s 8ms/step - loss: 1.8318 - accuracy: 0.2347 - val_loss: 1.7983 - val_accuracy: 0.2498
Epoch 2/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.8047 - accuracy: 0.2419 - val_loss: 1.7943 - val_accuracy: 0.2498
Epoch 3/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.7974 - accuracy: 0.2491 - val_loss: 1.7792 - val_accuracy: 0.2600
Epoch 4/30
1616/1616 [==============================] - 13s 8ms/step - loss: 1.7866 - accuracy: 0.2549 - val_loss: 1.7689 - val_accuracy: 0.2699
Epoch 5/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.7703 - accuracy: 0.2628 - val_loss: 1.7505 - val_accuracy: 0.2606
Epoch 6/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.7443 - accuracy: 0.2755 - val_loss: 1.7238 - val_accuracy: 0.2950
Epoch 7/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.6879 - accuracy: 0.3059 - val_loss: 1.6455 - val_ac

2022-09-02 17:10:20.885999: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1162980000 exceeds 10% of free system memory.


Epoch 1/30
1616/1616 [==============================] - 14s 8ms/step - loss: 1.8295 - accuracy: 0.2452 - val_loss: 1.7917 - val_accuracy: 0.2585
Epoch 2/30
1616/1616 [==============================] - 12s 7ms/step - loss: 1.7988 - accuracy: 0.2447 - val_loss: 1.7880 - val_accuracy: 0.2693
Epoch 3/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.7778 - accuracy: 0.2680 - val_loss: 1.7597 - val_accuracy: 0.2663
Epoch 4/30
1616/1616 [==============================] - 13s 8ms/step - loss: 1.7525 - accuracy: 0.2709 - val_loss: 1.7185 - val_accuracy: 0.2854
Epoch 5/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.6965 - accuracy: 0.2945 - val_loss: 1.6414 - val_accuracy: 0.3502
Epoch 6/30
1616/1616 [==============================] - 13s 8ms/step - loss: 1.6185 - accuracy: 0.3334 - val_loss: 1.6082 - val_accuracy: 0.3260
Epoch 7/30
1616/1616 [==============================] - 12s 8ms/step - loss: 1.5276 - accuracy: 0.3818 - val_loss: 1.4685 - val_ac